# Figures and Tables

## Table of Contents
1. [Read all parquets from Google Drive](#1-read-all-parquets-from-google-drive)
2. HadHeartDisease Summarization
3. Missing Threshold Representation
4. 

In [1]:
from asgmnt_2_tools import lazy_read_parquet
from great_tables import GT, md, html # Dataframe Formatting
import numpy as np # Array
import pandas as pd # Dataframe
import polars as pl # Lazyframe
import seaborn as sns # Plots
import sidetable as stb# Frequency/Missing Dataframe

## 1. Read All Parquets From Google Drive

| Dataframe Name | Description                        |
|----------------|------------------------------------|
| df_original    | No modifications to the data       |
| df             | Added column HadHearDisease. If a target variable is missing, HadHeartDisease missing.|
| df_heart_drop  | Drop all observations where a target variable is missing. Dropped `HadHeartAttack`, `HadStroke`, `HadAngina`, and `BMI`|
| df_heart_drop_## | Drop observations that pass the ## threshold for missing values |
| df*_imp        | Imputed version of all the above dataframes |

In [2]:
path_drive = "../../Data/GoogleDrive"

dict_lazy = dict(sorted(lazy_read_parquet(path_drive).items()))

## 2. Frequency of Adverse Cardiovascular Events

Describes how `HadHeartDisease` summarizes `HadHeartAttack`, `HadStroke`, and `HadAngina`

To save the table, install selenium using `pip install -U selenium` then
download chromedriver and put into `usr/local/bin`.

In [3]:
# Use SideTable to create frequency dataframe
tbl_1 = (
    dict_lazy["df"]
    .select(["HadHeartAttack", "HadStroke", "HadAngina", "HadHeartDisease"])
    .collect() # Lazyframe to dataframe
    .to_pandas() # Polars to Pandas
    # Using sidetable, generate a frequency table
    .stb.freq(
    ["HadHeartAttack", "HadStroke", "HadAngina", "HadHeartDisease"]
    )
    .drop(columns=['cumulative_count', 'cumulative_percent']) # Drop Cumulative Columns
    .assign(percent = lambda df: df['percent'] / 100) # Convert to Percent
)



In [4]:
tbl_1_cols = ["HadHeartAttack", "HadStroke", "HadAngina", "HadHeartDisease"]

# Transform all specified columns to nullify non-'Yes' values
tbl_1_tot = (
    dict_lazy["df"]
    .select(tbl_1_cols)
    .with_columns(
        [
            pl.when(pl.col(colname) == "Yes")
            .then(pl.lit(1))
            .otherwise(pl.lit(None))
            .alias(colname)
            for colname in tbl_1_cols
        ]
    )
    .count()
    .collect()
    .to_pandas()
)

In [5]:
tbl_1_no = (
    dict_lazy["df"]
    .select(tbl_1_cols)
    .with_columns(
        pl.when(
            (pl.col('HadHeartAttack') == "No") &
            (pl.col('HadStroke') == "No") &
            (pl.col('HadAngina') == "No")
        )
        .then(1)
        .otherwise(None)
        .alias('No_Sum')
        )
    .count()
    .sum()
    .collect()
    
)


pd.options.display.float_format = '{:,}'.format


tbl_1_tot['count'] = tbl_1_tot.iloc[0].sum()
tbl_1_tot['percent'] = tbl_1_tot.iloc[0].sum() / (tbl_1_no['No_Sum'][0] + tbl_1_tot.iloc[0].sum())
tbl_1_tot[' '] = 'Total Positive'



In [6]:
tbl_1 = pd.concat([tbl_1, tbl_1_tot], ignore_index=True)
tbl_1 = tbl_1.reindex(columns=[' ', "HadHeartAttack", "HadStroke", "HadAngina", "HadHeartDisease", 'count', 'percent'])

In [7]:

# Use GreatTables to format the dataframe
tbl_event_freq = (GT(tbl_1)
    .tab_header(title="Adverse Cardiovascular Event Frequency",
                subtitle = "Summarizing Adverse Cardiovascular Events with Heart Disease") # Title
    # Rename columns
    .cols_label(
        count = html("Count"),
        percent = html("Percent"),
        HadHeartAttack = html("Heart Attack"),
        HadStroke = html("Stroke"),
        HadAngina = html("Angina"),
        HadHeartDisease = html("Heart Disease")
    )
    .cols_align(align="center") # Body Alignment
    .fmt_percent(columns="percent") # Column Formatting
    .fmt_number(columns="count", decimals=0)
    .fmt_number(columns=tbl_1_cols,rows = 8, decimals=0)
    # Footnote
    .tab_source_note(
        source_note="Heart Disease is positive if at least Heart Attack, Stroke, and Angina were positive."
    )
)


# tbl_event_freq.save(file=f"./figures/tbl_event_freq.png")
tbl_event_freq

GT(_tbl_data=                  HadHeartAttack HadStroke HadAngina HadHeartDisease  \
0             NaN             No        No        No              No   
1             NaN             No        No       Yes             Yes   
2             NaN             No       Yes        No             Yes   
3             NaN            Yes        No        No             Yes   
4             NaN            Yes        No       Yes             Yes   
5             NaN            Yes       Yes       Yes             Yes   
6             NaN            Yes       Yes        No             Yes   
7             NaN             No       Yes       Yes             Yes   
8  Total Positive          25108     19239     26551           52256   

      count               percent  
0 387,696.0    0.8861420310392906  
1  12,438.0  0.028429064478526205  
2  11,939.0   0.02728851911956298  
3   9,789.0  0.022374345729240474  
4   9,259.0  0.021162944846974927  
5   2,568.0  0.005869580123882882  
6   2,091.0  0.004779319329843889  
7   1,730.0 0.0039541953326781105  
8 123,154.0   0.38849597163424837  , _body=<great_tables._gt_data.Body object at 0x16604e0e0>, _boxhead=Boxhead([ColInfo(var=' ', type=<ColInfoTypeEnum.default: 1>, column_label=' ', column_align='center', column_width=None), ColInfo(var='HadHeartAttack', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Heart Attack', type='html'), column_align='center', column_width=None), ColInfo(var='HadStroke', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Stroke', type='html'), column_align='center', column_width=None), ColInfo(var='HadAngina', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Angina', type='html'), column_align='center', column_width=None), ColInfo(var='HadHeartDisease', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Heart Disease', type='html'), column_align='center', column_width=None), ColInfo(var='count', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Count', type='html'), column_align='center', column_width=None), ColInfo(var='percent', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Percent', type='html'), column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=6, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=7, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=8, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title='Adverse Cardiovascular Event Frequency', subtitle='Summarizing Adverse Cardiovascular Events with Heart Disease', preheader=None), _stubhead=None, _source_notes=['Heart Disease is positive if at least Heart Attack, Stroke, and Angina were positive.'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x301bc51e0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x301bc5540>, <great_tables._gt_data.FormatInfo object at 0x301bc5720>, <great_tables._gt_data.FormatInfo object at 0x301bc58a0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value=

## HadHeartDisease Summarization

In [15]:
tbl_row = [] # to tabulate df rows

# If number of missing is > threshold, drop the observation.
for key, value in dict_lazy.items():
    if key[-2:].isdigit():
        threshold = int(key[-2:])
        tbl_row.append(
            {"Missing Threshold": threshold,
             "Number of Rows": value.select(pl.len()).collect().item()
             })  

tbl_row = pd.DataFrame(tbl_row)

(GT(tbl_row)
    .tab_header(title="observations Dropped",
                subtitle = "Summarizing Adverse Cardiovascular Events with Heart Disease") # Title
    .cols_align(align="center") # Body Alignment
    # Footnote
    .tab_source_note(
        source_note="Heart Disease is positive if at least Heart Attack, Stroke, and Angina were positive."
    )
)

GT(_tbl_data=   Missing Threshold  Number of Rows
0                  0          247367
1                  1          341472
2                  3          385494
3                  5          392561
4                 10          410633
5                 20          437240
6                 40          437510, _body=<great_tables._gt_data.Body object at 0x301c54af0>, _boxhead=Boxhead([ColInfo(var='Missing Threshold', type=<ColInfoTypeEnum.default: 1>, column_label='Missing Threshold', column_align='center', column_width=None), ColInfo(var='Number of Rows', type=<ColInfoTypeEnum.default: 1>, column_label='Number of Rows', column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=6, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title='observations Dropped', subtitle='Summarizing Adverse Cardiovascular Events with Heart Disease', preheader=None), _stubhead=None, _source_notes=['Heart Disease is positive if at least Heart Attack, Stroke, and Angina were positive.'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x301c54400>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, cate

## Missing Threshold Representation

## Todo

1. Table comparing the number of observations of `HadHeartAttack`, `HadStroke`, `HadAngina` and the summarized column `HadHeartDisease`.
    a. Provide an example using a rudimentary model (Logit Regression with enet regularization)

2. Table showing the number of observations after setting a missing threshold.
    a. Provide an example using a rudimentary model (Logit Regression with enet regularization)

3. Table comparing using imputation.